# PaW Toy Model — Page–Wootters Mechanism Demonstrator

**Paper**: *"The Observer as a Local Breakdown of Atemporality"* — G. Giani Moreno (2026)

This notebook implements the minimal demonstrator from Sections 5–6 and Appendix A.
All simulations are fully unitary — no Lindblad, no phenomenological decoherence.

**Important correction**: The paper writes $H_S = (\omega/2)\sigma_z$, but $\sigma_z$ generates only phase rotations, leaving $\langle\sigma_z\rangle$ constant for any initial state. We use $H_S = (\omega/2)\sigma_x$ with $|\psi_0\rangle = |0\rangle$ to obtain $\langle\sigma_z\rangle(t) = \cos(\omega t)$.

In [ ]:
# ── Section 1: Imports and Parameters ──────────────────────────────
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# Paper reference parameters
N = 30           # clock levels
dt = 0.2         # time step
omega = 1.0      # system frequency
g = 0.1          # S-E coupling strength
n_env_list = [2, 4, 6, 8]

# Output directory
os.makedirs("output", exist_ok=True)

# Matplotlib styling
plt.rcParams.update({
    'font.size': 11, 'axes.labelsize': 12,
    'axes.titlesize': 13, 'legend.fontsize': 10,
    'figure.dpi': 150,
})

print(f"QuTiP version: {qt.__version__}")
print(f"Parameters: N={N}, dt={dt}, ω={omega}, g={g}")
print(f"Theoretical oscillation period: {2*np.pi/(omega*dt):.1f} steps")

In [ ]:
# ── Section 2: Helper Functions ────────────────────────────────────

def build_H_total_SE(omega, g, n_env):
    """
    Build H_total = H_S + H_SE on the S ⊗ E Hilbert space.
    H_S  = (ω/2) σ_x ⊗ I_E
    H_SE = g Σ_j σ_x^(S) ⊗ σ_x^(E_j)

    Note: σ_z⊗σ_z coupling fails when E starts in |0⟩ (eigenstate of σ_z).
    σ_x⊗σ_x creates transitions since σ_x|0⟩ = |1⟩.
    """
    n = 1 + n_env
    # H_S
    ops = [qt.qeye(2)] * n
    ops[0] = qt.sigmax()
    H_S = (omega / 2) * qt.tensor(ops)
    # H_SE = g Σ_j σ_x^(S) ⊗ σ_x^(E_j)
    terms = []
    for j in range(n_env):
        ops = [qt.qeye(2)] * n
        ops[0] = qt.sigmax()
        ops[1 + j] = qt.sigmax()
        terms.append(g * qt.tensor(ops))
    H_SE = terms[0]
    for t in terms[1:]:
        H_SE = H_SE + t
    return H_S + H_SE

def run_version_a(N, dt, omega, full_history=False):
    """Version A: no environment. Returns (ks, sz, sz_theory)."""
    H_S = (omega / 2) * qt.sigmax()
    psi0 = qt.basis(2, 0)
    ks = np.arange(N)
    sz_th = np.cos(omega * ks * dt)
    sz = np.zeros(N)

    if full_history:
        cb = [qt.basis(N, k) for k in range(N)]
        parts = []
        for k in range(N):
            U = (-1j * H_S * k * dt).expm()
            parts.append(qt.tensor(cb[k], U * psi0))
        psi = parts[0]
        for p in parts[1:]:
            psi = psi + p
        psi = psi / np.sqrt(N)
        for k in range(N):
            proj = qt.tensor(cb[k] * cb[k].dag(), qt.qeye(2))
            c = proj * psi
            pk = c.norm()**2
            if pk > 1e-14:
                sz[k] = qt.expect(qt.sigmaz(), c.ptrace(1) / pk)
    else:
        for k in range(N):
            U = (-1j * H_S * k * dt).expm()
            sz[k] = qt.expect(qt.sigmaz(), U * psi0)
    return ks, sz, sz_th

def run_version_b(N, dt, omega, g, n_env):
    """Version B: with environment. Returns (ks, sz, s_eff, sz_th, fid)."""
    H = build_H_total_SE(omega, g, n_env)
    n = 1 + n_env
    psi0 = qt.tensor([qt.basis(2, 0)] * n)
    rho0_S = qt.basis(2, 0) * qt.basis(2, 0).dag()
    H_ideal = (omega / 2) * qt.sigmax()

    ks = np.arange(N)
    sz = np.zeros(N)
    s_eff = np.zeros(N)
    fid = np.zeros(N)
    sz_th = np.cos(omega * ks * dt)

    for k in range(N):
        tk = k * dt
        U = (-1j * H * tk).expm()
        psi_k = U * psi0
        rho_S = psi_k.ptrace(0)
        sz[k] = qt.expect(qt.sigmaz(), rho_S)
        s_eff[k] = qt.entropy_vn(rho_S)
        Uid = (-1j * H_ideal * tk).expm()
        rho_id = Uid * rho0_S * Uid.dag()
        fid[k] = qt.fidelity(rho_S, rho_id)**2
    return ks, sz, s_eff, sz_th, fid

def clock_back_action(N, dt):
    """Analytical ΔE_C(k) for orthogonal Salecker-Wigner clock."""
    wC = 2 * np.pi / (N * dt)
    ks = np.arange(N)
    return ks, wC * (ks - (N - 1) / 2)

print("Helper functions defined ✓")

## Version A — No Environment

Build the full PaW history state $|\Psi\rangle = \frac{1}{\sqrt{N}}\sum_k |k\rangle_C \otimes e^{-iH_S k\,dt}|0\rangle_S$ and condition on each clock tick to extract $\langle\sigma_z\rangle(k)$. Expected: clean sinusoid $\cos(\omega k\,dt)$ with period $\approx 31.4$ steps.

In [ ]:
# ── Version A: Full PaW history state construction ────────────────
ks_a, sz_a, sz_th_a = run_version_a(N, dt, omega, full_history=True)

print(f"Hilbert space dim: {N} × 2 = {N*2}")
print(f"Max |⟨σ_z⟩|: {np.max(np.abs(sz_a)):.6f}")
print(f"Max deviation from cos(ωkdt): {np.max(np.abs(sz_a - sz_th_a)):.2e}")
print(f"\nFirst 10 values:")
for k in range(10):
    print(f"  k={k:2d}  ⟨σ_z⟩={sz_a[k]:+.6f}  theory={sz_th_a[k]:+.6f}")

# ── Plot ──
fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(ks_a, sz_a, 'o-', color='#2196F3', markersize=5, lw=1.2,
        label=r'$\langle\sigma_z\rangle_k$ (PaW conditioned)')
ax.plot(ks_a, sz_th_a, '--', color='gray', alpha=0.7, lw=1.5,
        label=r'$\cos(\omega\, k\, dt)$ (theory)')
ax.axvline(2*np.pi/(omega*dt), color='green', ls=':', alpha=0.5,
           label=f'Period ≈ {2*np.pi/(omega*dt):.1f} steps')
ax.set_xlabel('k (clock tick)')
ax.set_ylabel(r'$\langle\sigma_z\rangle$')
ax.set_title('Version A: Coherent Oscillations (No Environment)')
ax.set_ylim(-1.15, 1.15)
ax.legend()
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig("output/version_A_oscillation.png", dpi=300)
plt.show()
print("→ Saved: output/version_A_oscillation.png")

## Version B — With Environment ($n_{env} = 4$)

Build $|\Psi\rangle = \frac{1}{\sqrt{N}}\sum_k |k\rangle_C \otimes U_{SE}(t_k)|0, 0000\rangle$ with $H_{SE} = g\sum_j \sigma_z^{(S)} \otimes \sigma_z^{(E_j)}$. Condition on clock, trace out environment, compute $\langle\sigma_z\rangle$ and $S_{\mathrm{eff}}$. All decoherence arises from partial trace — no Lindblad terms.

In [ ]:
# ── Version B: n_env = 4 ──────────────────────────────────────────
ne = 4
print(f"Running Version B with n_env={ne}...")
print(f"  Hilbert space (SE): 2 × 2^{ne} = {2 * 2**ne}")
print(f"  Full history state dim: {N} × {2 * 2**ne} = {N * 2 * 2**ne}")

ks_b, sz_b, seff_b, sz_th_b, fid_b = run_version_b(N, dt, omega, g, ne)

print(f"\nResults:")
print(f"  S_eff initial: {seff_b[0]:.4f}")
print(f"  S_eff final:   {seff_b[-1]:.4f}")
print(f"  S_eff max:     {np.max(seff_b):.4f}")
print(f"  Fidelity final: {fid_b[-1]:.4f}")

# ── Dual panel plot ──
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

ax1.plot(ks_b, sz_b, 'o-', color='#E91E63', markersize=4, lw=1,
         label=r'$\langle\sigma_z\rangle_k$ (with env)')
ax1.plot(ks_a, sz_a, '--', color='#2196F3', alpha=0.4, lw=1,
         label='Version A (no env)')
ax1.plot(ks_b, sz_th_b, '--', color='gray', alpha=0.3)
ax1.set_xlabel('k')
ax1.set_ylabel(r'$\langle\sigma_z\rangle$')
ax1.set_title(f'Damped Oscillations ($n_{{env}}={ne}$)')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(ks_b, seff_b, 's-', color='#FF5722', markersize=4, lw=1,
         label=r'$S_{\mathrm{eff}}(k)$')
ax2.axhline(np.log(2), color='gray', ls='--', alpha=0.5,
            label=r'$\ln 2 \approx 0.693$')
ax2.set_xlabel('k')
ax2.set_ylabel(r'$S_{\mathrm{eff}}$')
ax2.set_title('Effective Entropy Growth')
ax2.legend()
ax2.grid(True, alpha=0.3)

fig.suptitle(f'Version B: Fully unitary simulation, g={g}', fontsize=14, y=1.02)
fig.tight_layout()
fig.savefig("output/version_B_n4.png", dpi=300, bbox_inches='tight')
plt.show()
print("→ Saved: output/version_B_n4.png")

## Environment Size Scaling ($n_{env} \in \{2, 4, 6, 8\}$)

Run Version B for increasing environment sizes. Expect: stronger damping, more monotonic $S_{\mathrm{eff}}$ growth, and suppressed Poincaré recurrences as $d_E = 2^{n_{env}}$ increases.

In [ ]:
# ── Multi n_env sweep ─────────────────────────────────────────────
results = {}
colors = ['#4CAF50', '#2196F3', '#FF9800', '#9C27B0']

for ne in n_env_list:
    print(f"  n_env={ne} (dim SE={2*2**ne:5d}) ...", end=" ", flush=True)
    ks, sz, se, szt, f = run_version_b(N, dt, omega, g, ne)
    results[ne] = {'ks': ks, 'sz': sz, 's_eff': se, 'sz_th': szt, 'fid': f}
    print(f"S_eff final={se[-1]:.4f}, fidelity final={f[-1]:.4f}")

# ── S_eff overlay ──
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for i, ne in enumerate(n_env_list):
    r = results[ne]
    ax1.plot(r['ks'], r['s_eff'], 'o-', color=colors[i], markersize=3,
             label=f'$n_{{env}} = {ne}$  ($d_E = {2**ne}$)')
ax1.axhline(np.log(2), color='gray', ls='--', alpha=0.5)
ax1.set_xlabel('k (clock tick)')
ax1.set_ylabel(r'$S_{\mathrm{eff}}(k)$')
ax1.set_title('Informational Arrow: Entropy vs Environment Size')
ax1.legend()
ax1.grid(True, alpha=0.3)

for i, ne in enumerate(n_env_list):
    r = results[ne]
    ax2.plot(r['ks'], r['fid'], 'o-', color=colors[i], markersize=3,
             label=f'$n_{{env}} = {ne}$')
ax2.set_xlabel('k (clock tick)')
ax2.set_ylabel(r'$F^2(k)$')
ax2.set_title('Fidelity vs Ideal Schrödinger (Sec. 4.3)')
ax2.legend()
ax2.grid(True, alpha=0.3)

fig.tight_layout()
fig.savefig("output/env_scaling_comparison.png", dpi=300)
plt.show()
print("→ Saved: output/env_scaling_comparison.png")

## Back-Action Metric $\Delta E_C(k)$ (Sec. 4.2)

For the orthogonal Salecker–Wigner clock, $\Delta E_C(k) = \frac{2\pi}{N\,dt}(k - \frac{N-1}{2})$ is trivially linear. This metric becomes non-trivial for overlapping/quasi-ideal clock POVMs.

In [ ]:
# ── Back-action ΔE_C(k) ───────────────────────────────────────────
ks_ba, dE = clock_back_action(N, dt)

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(ks_ba, dE, 'o-', color='#607D8B', markersize=4)
ax.axhline(0, color='gray', ls='--', alpha=0.5)
ax.set_xlabel('k (clock tick)')
ax.set_ylabel(r'$\Delta E_C(k)$')
ax.set_title('Clock Back-Action (Sec. 4.2) — Linear for Orthogonal Readouts')
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig("output/back_action.png", dpi=300)
plt.show()
print(f"ΔE_C(0) = {dE[0]:+.4f},  ΔE_C({N-1}) = {dE[-1]:+.4f}")

## CSV Export — Tables for Paper

Export numerical results as CSV files for inclusion in the paper.

In [ ]:
# ── Table 1: Version A ────────────────────────────────────────────
df_a = pd.DataFrame({
    'k': ks_a,
    't_k': ks_a * dt,
    'σz_numerical': np.round(sz_a, 6),
    'σz_analytic': np.round(sz_th_a, 6),
    'deviation': np.round(np.abs(sz_a - sz_th_a), 2e-10),
})
df_a.to_csv("output/table_version_A.csv", index=False)
print("Table 1 — Version A (first 10 rows):")
print(df_a.head(10).to_string(index=False))

# ── Table 2: Version B detail (all n_env) ────────────────────────
frames = []
for ne in n_env_list:
    r = results[ne]
    df = pd.DataFrame({
        'k': r['ks'],
        'n_env': ne,
        'σz': np.round(r['sz'], 6),
        'S_eff': np.round(r['s_eff'], 6),
        'fidelity_sq': np.round(r['fid'], 6),
    })
    frames.append(df)
df_b = pd.concat(frames, ignore_index=True)
df_b.to_csv("output/table_version_B_all.csv", index=False)
print(f"\nTable 2 — Version B ({len(df_b)} rows) saved")

# ── Table 3: Summary statistics ──────────────────────────────────
summary_rows = []
for ne in n_env_list:
    r = results[ne]
    amp_last10 = np.max(np.abs(r['sz'][-10:]))
    summary_rows.append({
        'n_env': ne,
        'd_E': 2**ne,
        'S_eff_initial': round(r['s_eff'][0], 4),
        'S_eff_final': round(r['s_eff'][-1], 4),
        'S_eff_max': round(np.max(r['s_eff']), 4),
        'σz_amp_last10': round(amp_last10, 4),
        'fidelity_final': round(r['fid'][-1], 4),
    })
df_summary = pd.DataFrame(summary_rows)
df_summary.to_csv("output/table_summary.csv", index=False)

print("\nTable 3 — Summary:")
print(df_summary.to_string(index=False))
print("\n✓ All CSV files saved to output/")

## Publication Figure — Multi-Panel Summary

Combined figure for paper: (a) Version A oscillations, (b) Version B damped oscillations, (c) $S_{\mathrm{eff}}$ scaling with $n_{env}$, (d) Fidelity degradation.

In [ ]:
# ── Publication multi-panel figure ─────────────────────────────────
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# (a) Version A
ax = axes[0, 0]
ax.plot(ks_a, sz_a, 'o-', color='#2196F3', markersize=4, lw=1)
ax.plot(ks_a, sz_th_a, '--', color='gray', alpha=0.6)
ax.set_xlabel('k')
ax.set_ylabel(r'$\langle\sigma_z\rangle$')
ax.set_title('(a) Version A: Coherent Oscillations')
ax.set_ylim(-1.15, 1.15)
ax.grid(True, alpha=0.2)

# (b) Version B (n_env=4)
ax = axes[0, 1]
r4 = results[4]
ax.plot(r4['ks'], r4['sz'], 'o-', color='#E91E63', markersize=3, lw=1)
ax.plot(r4['ks'], r4['sz_th'], '--', color='gray', alpha=0.4)
ax.set_xlabel('k')
ax.set_ylabel(r'$\langle\sigma_z\rangle$')
ax.set_title(r'(b) Version B: Damped ($n_{env}=4$)')
ax.grid(True, alpha=0.2)

# (c) S_eff scaling
ax = axes[1, 0]
for i, ne in enumerate(n_env_list):
    r = results[ne]
    ax.plot(r['ks'], r['s_eff'], 'o-', color=colors[i], markersize=3,
            label=f'$n_{{env}}={ne}$')
ax.axhline(np.log(2), color='gray', ls='--', alpha=0.4)
ax.set_xlabel('k')
ax.set_ylabel(r'$S_{\mathrm{eff}}(k)$')
ax.set_title(r'(c) Informational Arrow')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.2)

# (d) Fidelity
ax = axes[1, 1]
for i, ne in enumerate(n_env_list):
    r = results[ne]
    ax.plot(r['ks'], r['fid'], 'o-', color=colors[i], markersize=3,
            label=f'$n_{{env}}={ne}$')
ax.set_xlabel('k')
ax.set_ylabel(r'$F^2(k)$')
ax.set_title(r'(d) Fidelity vs Ideal Dynamics')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.2)

fig.suptitle(f'PaW Toy Model Summary  (N={N}, dt={dt}, ω={omega}, g={g})',
             fontsize=14, y=1.01)
fig.tight_layout()
fig.savefig("output/paper_figure_summary.png", dpi=300, bbox_inches='tight')
fig.savefig("output/paper_figure_summary.pdf", bbox_inches='tight')
plt.show()
print("→ Saved: output/paper_figure_summary.png")
print("→ Saved: output/paper_figure_summary.pdf")